### Importing necessary libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import PIL
from PIL import Image

# import seaborn as sns

In [4]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

In [5]:
from keras.layers import MaxPool2D, Conv2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
original_training_df = pd.read_csv("drive/MyDrive/train_hackathon/train.csv")
original_test_df= pd.read_csv("drive/MyDrive/test_hackathon/test.csv")

In [9]:
original_training_df.head()

,image_id,filename,label
0,1,1.jpg,0
1,2,2.jpg,0
2,3,3.jpg,0
3,4,4.jpg,0
4,5,5.jpg,0


In [10]:
original_training_df.shape

(8079, 3)

In [11]:
train_df, val_df = train_test_split(original_training_df, test_size=0.2, random_state=42, stratify= original_training_df["label"])

In [12]:
# I have to cast the labels because model.compile complained about int64. it expects float32 (for some reason)
# train_df["label"] = train_df["label"].astype('float32')
# val_df["label"] = val_df["label"].astype('float32')

In [13]:
train_df["label"].value_counts()

0    6091
1     372
Name: label, dtype: int64

In [14]:
val_df["label"].value_counts()

0    1523
1      93
Name: label, dtype: int64

In [15]:
train_df.shape, val_df.shape

((6463, 3), (1616, 3))

# **loading the images into a tensorflow dataset**


QUESTION:
Is there a smarter way than the one I used to load the images if the folder contains images from both classes and we have a csv
file that tells us the label of each image?





There is this method that works if the directory has subdirectories each corresponding to a class: </br>
dir= "drive/MyDrive/train_hackathon/resized_images/" </br>
dataset= tf.keras.preprocessing.image_dataset_from_directory(dir)

If the directory doesn't have subdirectories, it gives a ValueError: No images found in directory
drive/MyDrive/train_hackathon/resized_images/. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [16]:
#dir = "drive/MyDrive/train_hackathon/resized_images/"
training_dir = "drive/MyDrive/train_hackathon/images/"

def read_image(image_name, label):
  image= tf.io.read_file(training_dir + image_name) # returns an encoded image
  image = tf.image.decode_image(image, dtype= tf.float32, expand_animations = False) # returns a tensorflow.python.framework.ops.EagerTensor
  return image, label

**Very Important:** in the **tf.image.decode_image** we should pass as a parameter **expand_animations= False**, otherwise we will get errors applying tf.image.resize. The error was:   
File "<ipython-input-25-6ff5a1ab44c3>", line 2, in trivial_preprocessing  *
        image = tf.image.resize(image, [height, width])

    ValueError: 'images' contains no shape.

Here is the documentation: https://www.tensorflow.org/api_docs/python/tf/io/decode_image

Stackoverflow: https://stackoverflow.com/questions/44942729/tensorflowvalueerror-images-contains-no-shape


In [17]:
images_names_train = train_df["filename"].values
labels_train = train_df["label"].values
ds_train = tf.data.Dataset.from_tensor_slices((images_names_train, labels_train))
ds_train = ds_train.map(read_image)

In [18]:
ds_train

<_MapDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [19]:
# same for validation data
images_names_val = val_df["filename"].values
labels_val = val_df["label"].values
ds_val = tf.data.Dataset.from_tensor_slices((images_names_val, labels_val))
ds_val = ds_val.map(read_image)

In [20]:
ds_train = ds_train.cache()
ds_val = ds_val.cache()

In [21]:
len(ds_train)

6463

In [22]:
type(ds_train)

tensorflow.python.data.ops.cache_op.CacheDataset

In [23]:
ds_train

<CacheDataset element_spec=(TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [24]:
img = read_image("1.jpg",0)[0]

In [25]:
type(img)

tensorflow.python.framework.ops.EagerTensor

In [26]:
img.get_shape(), img.shape

(TensorShape([649, 866, 3]), TensorShape([649, 866, 3]))

# Trivial preprocessing: resizing and rescaling

## Attempt 1:

In [27]:
height = 600
width = 800

In [28]:
def trivial_preprocessing(image,label):
      image = tf.image.resize(image, [height, width])
      image = tf.ensure_shape(image, [height, width, 3])
      image = tf.cast(image, tf.float32)/255.0
      return image,label

In [29]:
# ds_train = ds_train.shuffle(buffer_size = len(ds_train)).map(trivial_preprocessing).batch(128)
# ds_val= ds_val.shuffle(buffer_size = len(ds_val)).map(trivial_preprocessing).batch(128)

ds_train = ds_train.map(trivial_preprocessing)

In [30]:
ds_train

<_MapDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [31]:
ds_train.cache()

<CacheDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

**To understand .shuffle and .batch** watch this https://www.youtube.com/watch?v=c7G5W4Wv72Q

## Attempt 2:

In [ ]:
height = 600
width = 800

trivial_preprocessing_layers = tf.keras.Sequential([
  layers.Resizing(height= height, width= width),
  # Ensure the image has 3 channels (e.g., for RGB images)
  layers.Lambda(lambda x: tf.ensure_shape(x, [height, width, 3])),
  layers.Rescaling(1./255)
])

In [ ]:
ds_train = ds_train.map(lambda image, label: (trivial_preprocessing_layers(image), label))

In [ ]:
ds_train

<_MapDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

# Baseline model without data augmentation

In [ ]:
baseline_model = Sequential()
baseline_model.add(InputLayer(input_shape=(height, width, 3)))
baseline_model.add(Conv2D(5, (5, 5), activation='relu', strides=(2,2), padding='valid'))
baseline_model.add(MaxPool2D(pool_size=(5, 5), padding='same'))
baseline_model.add(Flatten())
baseline_model.add(Dense(units=128, activation='sigmoid'))
baseline_model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
baseline_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 398, 5)       380       
                                                                 
 max_pooling2d (MaxPooling2  (None, 60, 80, 5)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 24000)             0         
                                                                 
 dense (Dense)               (None, 128)               3072128   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3072637 (11.72 MB)
Trainable params: 3072637 (11.72 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
metric = tf.keras.metrics.F1Score(average= "macro", threshold= 0.5, name='macro_f1_score')
adam = Adam(learning_rate =1e-5)
baseline_model.compile(optimizer = adam, loss='binary_crossentropy', metrics=[metric])

In [ ]:
train_df.dtypes

image_id      int64
filename     object
label       float32
dtype: object

In [ ]:
num_epochs = 50
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, mode='min')
baseline_model_history=  baseline_model.fit(ds_train, epochs=num_epochs, validation_data=ds_val, callbacks=[early_stopping])

Epoch 1/50
51/51 [==============================] - 1731s 33s/step - loss: 0.5145 - macro_f1_score: 0.0000e+00 - val_loss: 0.4972 - val_macro_f1_score: 0.0000e+00
Epoch 2/50
51/51 [==============================] - 549s 11s/step - loss: 0.4744 - macro_f1_score: 0.0000e+00 - val_loss: 0.4457 - val_macro_f1_score: 0.0000e+00
Epoch 3/50
51/51 [==============================] - 538s 11s/step - loss: 0.4140 - macro_f1_score: 0.0000e+00 - val_loss: 0.3782 - val_macro_f1_score: 0.0000e+00
Epoch 4/50
51/51 [==============================] - 518s 10s/step - loss: 0.3463 - macro_f1_score: 0.0000e+00 - val_loss: 0.3140 - val_macro_f1_score: 0.0000e+00
Epoch 5/50
51/51 [==============================] - 533s 10s/step - loss: 0.2912 - macro_f1_score: 0.0000e+00 - val_loss: 0.2698 - val_macro_f1_score: 0.0000e+00
Epoch 6/50
51/51 [==============================] - 502s 10s/step - loss: 0.2565 - macro_f1_score: 0.0000e+00 - val_loss: 0.2446 - val_macro_f1_score: 0.0000e+00
Epoch 7/50
51/51 [=========

KeyboardInterrupt: 

**Note:** The positive class is the minority class; class of fraud images. Usually in binary classification problems, the 1 class is considered to be the positive class.

**problem: notice the 0 values for the macro f1 score. It doesn't make sense that macro f1 score = 0. What is shown is actually the f1 score for the minority class only despite specifying "macro" when defining the f1 metric. Let us check this by checking the classification report after training the baseline model for one epoch only.**

In [ ]:
baseline_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=[metric])
baseline_model.fit(ds_train, epochs=1)
val_results = baseline_model.evaluate(ds_val)
print("On the validation set: ", "val loss = ", val_results[0], "Validation Macro F1 Score = ", val_results[1])

51/51 [==============================] - 50s 967ms/step - loss: 0.2209 - macro_f1_score: 0.0000e+00
On the validation set:  val loss =  0.2209019809961319 Validation Macro F1 Score =  0.0


In [ ]:
val_pred = baseline_model.predict(ds_val) # predictions are floats (probabilities of belonging to the positive class)

val_true_labels = []
for _, label in ds_val.unbatch().as_numpy_iterator():
    val_true_labels.append(label)

# val_true_labels is a list of floats.

51/51 [==============================] - 52s 1s/step


'\nprint("Classification Report:")\nprint(classification_report(val_true_labels, val_pred.argmax(axis=1)))\n'

In [ ]:
classification_threshold = 0.5
val_pred_binary = (val_pred > classification_threshold).astype(int)

In [ ]:
val_true_labels = [int(label) for label in val_true_labels]

In [ ]:
print(classification_report(val_true_labels, val_pred_binary))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1523
           1       0.00      0.00      0.00        93

    accuracy                           0.94      1616
   macro avg       0.47      0.50      0.49      1616
weighted avg       0.89      0.94      0.91      1616



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
f1_score(val_true_labels, val_pred_binary, average='macro')

0.4851863650844218

**Workaround: Define a custom callback**

In [ ]:
class MacroF1ScoreCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, ds_train, ds_val):
        super(MacroF1ScoreCallback, self).__init__()
        self.model = model
        self.ds_train = ds_train
        self.ds_val = ds_val
        self.macro_f1_train = []
        self.macro_f1_val = []

    def calculate_macro_f1(self, dataset):
        y_true = tf.concat([y for x, y in dataset], axis=0)
        y_pred_probs = self.model.predict(dataset)
        y_pred = (y_pred_probs > 0.5).astype(int)
        macro_f1 = f1_score(y_true.numpy(), y_pred, average='macro')
        return macro_f1

    def on_epoch_end(self, epoch, logs=None):
        macro_f1_train = self.calculate_macro_f1(self.ds_train)
        self.macro_f1_train.append(macro_f1_train)
        macro_f1_val = self.calculate_macro_f1(self.ds_val)
        self.macro_f1_val.append(macro_f1_val)
        print(f'Epoch {epoch + 1} - Macro F1 Score (Train): {macro_f1_train}')
        print(f'Epoch {epoch + 1} - Macro F1 Score (Validation): {macro_f1_val}')

macro_f1_callback = MacroF1ScoreCallback(model=baseline_model, ds_train=ds_train, ds_val=ds_val)

In [ ]:
f1_metric = tf.keras.metrics.F1Score(average=None, threshold= 0.5, name='f1_score')
baseline_model.compile(optimizer=adam, loss='binary_crossentropy', metrics=[f1_metric])

# Fit the model with the custom callback
baseline_model.fit(ds_train, epochs=1, validation_data=ds_val, callbacks=[macro_f1_callback])

51/51 [==============================] - 50s 965ms/step
Epoch 1 - Macro F1 Score (Train): 0.4851840050979767
Epoch 1 - Macro F1 Score (Validation): 0.4851863650844218
51/51 [==============================] - 913s 18s/step - loss: 0.2209 - f1_score: 0.0000e+00 - val_loss: 0.2208 - val_f1_score: 0.0000e+00


# **Image Augmentation**

**Reminder: We augment training data only!**

### Augmentation functions

In [50]:
seed = tf.constant([random.randint(0, 9999), 0], dtype=tf.int32)

In [51]:
def add_noise(img, noise_factor=0.2):
    noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_factor, dtype=tf.float32)
    noisy_image = img + noise
    noisy_image = tf.clip_by_value(noisy_image, 0.0, 1.0)
    return noisy_image

def flip_left_right(image):
  return tf.image.flip_left_right(image)

def random_hue(image):
  return tf.image.stateless_random_hue(image, max_delta= 0.469, seed= seed)

def random_saturation(image):
  return tf.image.stateless_random_saturation(image, lower=0.469, upper=90, seed = seed)

#def random_quality(image):
 # return tf.image.stateless_random_jpeg_quality(image, min_jpeg_quality=60, max_jpeg_quality=90, seed= seed)

**Note that the seed is obligatory in tf.image functions whereas it is an optional argument in the tf.keras layers.** <br/>
**I don't understand why it should have two elements.** <br/>
A tf.image function with the same seed always outputs the same augmented image.

In [32]:
random_brightness = tf.keras.layers.RandomBrightness(factor= 0.8, value_range=[0.0, 1.0], seed=None)
random_contrast = tf.keras.layers.RandomContrast(factor=0.5, seed=None)
random_translation = tf.keras.layers.RandomTranslation(height_factor= 0.5,width_factor=0.5,fill_mode='reflect',interpolation='bilinear',seed=None)
random_rotation = tf.keras.layers.RandomRotation(factor=0.3, fill_mode='reflect',interpolation='bilinear',seed=None)
random_zoom = tf.keras.layers.RandomZoom(height_factor= (-0.3, -0.2), width_factor=None,fill_mode='reflect',interpolation='bilinear',seed=None)

In [52]:
augmentation_functions= [add_noise, flip_left_right, random_hue, random_saturation, random_brightness, random_contrast,
                        random_translation,  random_rotation, random_zoom]

In [53]:
len(augmentation_functions)

9

### Attempt 1:

In [42]:
def augment(image, label):

    augmented_images = [image]  # Should I add the original image to this list?

    if label == 1:
        #for func in augmentation_functions:
         #     augmented_images.append(func(image))
        augmented_images.append((random_contrast(image),label))

    for aug_img in augmented_images:
        yield aug_img, label

    # return augmented_images, tf.repeat(label, len(augmented_images))
    # returns a tuple with two elements. The first is a list of 10 tensor images and the second is a tensor with the 10 labels.

*   We could've used ***return augmented_images, tf.repeat(label, len(augmented_images))*** which returns a tuple with two elements. The first is a list of 10 tensor images and the second is a tensor with the 10 labels. So far so good. The problem is when we apply the ds_train.map(augment) which will add to ds_train this tuple representing 10 images together which will be considered an element of ds_train. The usual element of ds_train, however, is a tuple with one image and one label only. This will make the tf.keras.layers confuse the shapes of the input images and lead to ValueError that the tensor shape is unknown.
*   The yield keyword returns a generator which can be used to create an augmented dataset using tf.dataset.Data.from_generator. We can then concatenated the augmented dataset with ds_train.


*   I want to use flatmap. Maybe something like this: **.flat_map(lambda x, y: tf.data.Dataset.from_generator(lambda: augment(x, y), output_signature=(tf.TensorSpec(shape=(None, None, None), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int32))))**?

In [46]:
ds_train_augmented = ds_train.flat_map(lambda image, label: tf.data.Dataset.from_generator(
    augment, output_signature=(tf.TensorSpec(shape=(600, 800, 3), dtype=tf.float32), tf.TensorSpec(shape=(), dtype=tf.int64)), args=(image, label)))

In [47]:
len(ds_train_augmented)

TypeError: The dataset length is unknown.

In [48]:
tf.data.experimental.cardinality(ds_train_augmented).numpy()

-2

### Attempt 2: Create a list of augmented images, create a tensorflow dataset from it, and then concatenate it with the original dataset.

In [ ]:
augmented_images = []
count = 0
for image, label in ds_train:
  if label == 1:
        aug_img = random_contrast(image)
        augmented_images.append((aug_img, label))
        # count = count + 1
        # if count >= 2:
          #   break

In [88]:
augmented_ds_from_slices = tf.data.Dataset.from_tensor_slices(
    (tf.stack([img for img, _ in augmented_images]),
     tf.stack([lbl for _, lbl in augmented_images]))
)

In [89]:
augmented_ds_from_slices

<_TensorSliceDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [109]:
len(augmented_ds_from_slices)

2

In [90]:
ds_train_augmented_from_slices = ds_train.concatenate(augmented_ds_from_slices)

To check the size of a tensorflow dataset:

In [110]:
len(ds_train_augmented_from_slices)

6465

In [111]:
tf.data.experimental.cardinality(ds_train_augmented_from_slices).numpy()

6465

using tf.data.Dataset.from_generator

In [78]:
def image_label_generator():
    for img, lbl in augmented_images:
        yield img, lbl

augmented_ds_from_gen = tf.data.Dataset.from_generator(
    image_label_generator,
    output_signature=(
        tf.TensorSpec(shape=(height, width, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int64)
    )
)

In [79]:
augmented_ds_from_gen

<_FlatMapDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [80]:
ds_train

<_MapDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [81]:
ds_train_augmented_from_gen = ds_train.concatenate(augmented_ds_from_gen)

### Attempt 3: Creating an empty tensorflow dataset and adding to it augmented images one by one and then once it is ready we concatenate it with the original dataset.


In [101]:
input_shape = (600, 800, 3)

# Create empty tensors
empty_input = tf.Variable(tf.zeros(input_shape, dtype=tf.float32), trainable=False)
empty_label = tf.Variable(tf.zeros((), dtype=tf.int64), trainable=False)
empty_data = (empty_input, empty_label)

augmented_ds = tf.data.Dataset.from_tensors(empty_data)

count=0
for image, label in ds_train:
    if label == 1:
            augmented_image = random_contrast(image)
            empty_input.assign(augmented_image)
            empty_label.assign(label)
            augmented_ds = augmented_ds.concatenate(tf.data.Dataset.from_tensors((empty_input, empty_label)))
            count = count + 1
    if count >= 2:
            break

In [102]:
augmented_ds

<_ConcatenateDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [103]:
ds_train_augmented = ds_train.concatenate(augmented_ds)

In [104]:
ds_train_augmented

<_ConcatenateDataset element_spec=(TensorSpec(shape=(600, 800, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

How to save and load a NN model?
https://machinelearningmastery.com/save-load-keras-deep-learning-models/ <br/>
The model after loading might give different results: https://machinelearningmastery.com/different-results-each-time-in-machine-learning/